In [3]:
from sqlconnector import SQLConnector
import os
import pandas as pd

machineJsonFile = 'machines.json'
accessoriesJsonFile = 'machine-accessories.json'

firebaseImportPrg = 'C:\\Users\\loser.ralph\\workspace\\mhm-firebase-import'
databaseFilePath = 'database-files'
importPrg = 'json-to-firestore.js'

In [4]:

sqlConnector = SQLConnector('copyManager', '|~qpc,wm*of/=%"$i7h!i; m2~i4vab^')
conn = sqlConnector.create_connection()

## Artikel Abfrage
articleQuery = 'select AATR.[No_] as \'itemNo\',ART.[Description] as \'descrGER\',ENU.[Description] as \'descrENG\',ART.[Base Unit of Measure] as \'quantity\',BATR.[Name] as \'machineFamily\',CATR.[Value] as \'sort\', ART.[Unit Cost] as \'listPrice\' from [MHM$Item Attribute Value Mapping] as AATR join [MHM$Item Attribute] as BATR on AATR.[Item Attribute ID] = BATR.[ID] join [MHM$Item Attribute Value] as CATR on CATR.[Attribute ID] = AATR.[Item Attribute ID] join [MHM$Item] as ART on AATR.[No_] = ART.[No_] join [MHM$Item Translation] as ENU on AATR.[No_] = ENU.[Item No_] where (AATR.[Item Attribute ID] < 39) and (ENU.[Language Code] = \'ENU\') and (AATR.[Item Attribute Value ID] = CATR.[ID]) order by AATR.[No_] asc'

df = pd.read_sql(articleQuery, conn)
print(df.head())
machine_accessory = df[df['machineFamily'].str.contains("Maschine Zubehör")]
machine_accessory.set_index('itemNo', inplace=True)

          itemNo                             descrGER  \
0  20-0-0-0017-0        Rakelaufnahmeschiene komplett   
1  20-0-0-0096-0  Siebaufnahme U-Profil Y-Achse 50x70   
2  20-0-0-0096-0  Siebaufnahme U-Profil Y-Achse 50x70   
3  20-0-0-0096-0  Siebaufnahme U-Profil Y-Achse 50x70   
4  20-0-0-0096-0  Siebaufnahme U-Profil Y-Achse 50x70   

                      descrENG quantity                   machineFamily sort  \
0  squeegee pivot bar complete       ST            Maschine SP5000 AUTO    1   
1  U-profile adapter bar 50x70       ST         Maschine SP5000 Manuell    1   
2  U-profile adapter bar 50x70       ST            Maschine SP5000 AUTO    1   
3  U-profile adapter bar 50x70       ST  Maschine SP5000 Manuell VAKUUM    1   
4  U-profile adapter bar 50x70       ST     Maschine SP5000 AUTO VAKUUM    1   

   listPrice  
0    9.26041  
1   28.64950  
2   28.64950  
3   28.64950  
4   28.64950  


In [5]:
#negated
#Machine accessories with machine family as array
machines = df[~df['machineFamily'].str.contains("Maschine Zubehör")]

grouped_machine_family = machines.groupby(['itemNo']).machineFamily.apply(';'.join).to_frame()

accessories = df[df['machineFamily'].str.contains("Maschine Zubehör")]

merged = pd.merge(accessories, grouped_machine_family, on=['itemNo'])
merged.drop('machineFamily_x', axis=1, inplace=True)
merged.rename(columns={'machineFamily_y': 'machineFamily'}, inplace=True)
merged['machineFamily'] = merged['machineFamily'].str.replace('Maschine ', '')
merged['listPrice'] = merged['listPrice'].round(2)

In [6]:
import json
res = json.loads(merged.to_json(orient='records'))
with open(firebaseImportPrg + os.sep + databaseFilePath + os.sep + accessoriesJsonFile, 'w') as outfile:
    json.dump(res, outfile)

In [7]:
####################################################################################
########################### M a c h i n e s ########################################
####################################################################################

In [8]:
machines = df[df['itemNo'].str.startswith('S')]
machines = machines.rename(columns={'sort': 'tableNo'})

machines['machineFamily'] = machines['machineFamily'].str.replace('Maschine ', '')

machineFamilies = machines[~machines['machineFamily'].str.match('(Anzahl Tische)|[0-9]{2,3}x[0-9]{2,3}')][['itemNo', 'machineFamily']]
machineFamilies = machineFamilies.set_index('itemNo')

In [9]:
tableNumbers = machines[machines['machineFamily'].str.contains('Anzahl Tische')][['itemNo', 'tableNo']]
tableNumbers = tableNumbers.set_index('itemNo')

In [10]:
printSizes = df[df['itemNo'].str.startswith('S')]
printSizes['machineFamily'] = printSizes['machineFamily'].str.replace('Maschine ', '')
printSizes.rename(columns={'machineFamily': 'printSize'}, inplace=True)
printSizes = printSizes[printSizes['printSize'].str.match('[0-9]{2,3}x[0-9]{2,3}')][['itemNo', 'printSize']]
printSizes.set_index('itemNo', inplace=True)

C:\Users\LOSER~1.RAL\AppData\Local\Temp/ipykernel_32888/4171649076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  printSizes['machineFamily'] = printSizes['machineFamily'].str.replace('Maschine ', '')
C:\Python310\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [11]:
machines.drop('machineFamily', inplace=True, axis=1)
machines2 = pd.merge(machines, machineFamilies,  on=['itemNo'])
machines2 = pd.merge(machines2, printSizes, on=['itemNo'])

In [12]:
machines2.drop('tableNo', inplace=True, axis=1)
machines3 = pd.merge(machines2, tableNumbers, on=['itemNo'])


In [13]:
machines3 = machines3.drop_duplicates(subset=['itemNo'])
res = json.loads(machines3.to_json(orient='records'))
print(os.path)
with open(firebaseImportPrg + os.sep + databaseFilePath + os.sep + machineJsonFile, 'w') as outfile:
    json.dump(res, outfile)

<module 'ntpath' from 'C:\\Python310\\lib\\ntpath.py'>


In [14]:
import subprocess
import_process = subprocess.Popen(['node', firebaseImportPrg + os.sep + importPrg], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout,stderr = import_process.communicate()

stderr

b''